
Привет, меня зовут Люман Аблаев. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на красные комментарии ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:


<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ...
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>







<font color='orange' style='font-size:24px; font-weight:bold'>Общее впечатление</font>
* Спасибо за очень качественную работу - видно, что вложено много труда.
- Я оставил некоторые советы, надеюсь они будут полезными и интересными
- Есть некоторые недочеты, которые нужно поправить, но у тебя это не должно занять много времени)
- Жду обновленную работу





<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Привет, Люман! Спасибо за комментарии. Постаралась поправить (не все синие комментарии успела) и добавила комменты про изменения.
</div>

<font color='orange' style='font-size:24px; font-weight:bold'>Общее впечатление[2]</font>
* Спасибо за доработки
- Приятно видеть фидбек 
- Недочеты исправлены - работа полностью корректна
- Рад был быть полезен. Удачи в дальнейшем обучении!


# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

In [1]:
#!pip install phik ydata_profiling shap numpy==1.23 numba==0.58.1 -q

In [2]:
#!pip install --upgrade pip seaborn ydata_profiling Pillow scikit-learn scipy -q

In [3]:
#!pip install --upgrade catboost

In [4]:
import os
import re
import time

import lightgbm as lgb
import numpy as np
import pandas as pd
import ydata_profiling

from catboost import CatBoostRegressor
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, TargetEncoder, OrdinalEncoder
from lightgbm import LGBMRegressor


<div class="alert alert-block alert-success">
<b>Успех:</b> Импорты  на месте
</div>


## Подготовка данных

<b> Описание данных: </b> 

Данные находятся в файле `/datasets/autos.csv`
  
Признаки:  
`DateCrawled` — дата скачивания анкеты из базы  
`VehicleType` — тип автомобильного кузова  
`RegistrationYear` — год регистрации автомобиля  
`Gearbox` — тип коробки передач  
`Power` — мощность (л. с.)  
`Model` — модель автомобиля  
`Kilometer` — пробег (км)  
`RegistrationMonth` — месяц регистрации автомобиля  
`FuelType` — тип топлива  
`Brand` — марка автомобиля  
`Repaired` — была машина в ремонте или нет  
`DateCreated` — дата создания анкеты  
`NumberOfPictures` — количество фотографий автомобиля  
`PostalCode` — почтовый индекс владельца анкеты (пользователя)  
`LastSeen` — дата последней активности пользователя  
  
Целевой признак  
`Price` — цена (евро)  

In [5]:
# Считаем CSV-файл с данными с помощью библиотеки pandas в датафрейм и изучим общую информацию

path = '/datasets'

try:
    # Чтение датафреймов
    autos = pd.read_csv(os.path.join(path, 'autos.csv'), parse_dates=['DateCrawled', 'DateCreated', 'LastSeen'])

except FileNotFoundError:
    print('Один или несколько файлов не найдены.')

In [6]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   DateCrawled        354369 non-null  datetime64[ns]
 1   Price              354369 non-null  int64         
 2   VehicleType        316879 non-null  object        
 3   RegistrationYear   354369 non-null  int64         
 4   Gearbox            334536 non-null  object        
 5   Power              354369 non-null  int64         
 6   Model              334664 non-null  object        
 7   Kilometer          354369 non-null  int64         
 8   RegistrationMonth  354369 non-null  int64         
 9   FuelType           321474 non-null  object        
 10  Brand              354369 non-null  object        
 11  Repaired           283215 non-null  object        
 12  DateCreated        354369 non-null  datetime64[ns]
 13  NumberOfPictures   354369 non-null  int64   

In [7]:
autos.sample(3)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
159901,2016-03-11 11:59:22,2800,suv,2001,manual,120,pajero,100000,12,NaN,mitsubishi,no,2016-03-11,0,74357,2016-03-11 11:59:22
106542,2016-03-16 17:45:33,1650,NaN,2018,manual,105,golf,150000,5,petrol,volkswagen,NaN,2016-03-16,0,46236,2016-03-17 15:46:43
180433,2016-04-01 15:48:30,1999,bus,1999,manual,122,other,150000,12,petrol,mitsubishi,no,2016-04-01,0,61130,2016-04-07 13:50:05


In [8]:
autos.head(3)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14,0,90480,2016-04-05 12:47:46


<b>По результатам загрузки</b> видим, что полученные данные соответствуют их описанию.

In [9]:
#ydata_profiling.ProfileReport(autos)

<b>По результатам:</b>
- цена: 3% - нулевые значения, пропусков нет, максимум - 20000, среднее - 4416, больше всего зависит от мощности и года регистрации
- дата скачивания анкеты: с 05/03/2016 по 07/04/2016, нет пропущенных значений,
- тип кузова: больше всего в данных - седанов, всего 8 категорий, 10% значений пропущено,
- год регистрации: нет пропусков, но есть странные значения - год 1000 и 9999,
- коробка: 2 варианта, 5% пропусков,
- мощность: нет пропусков, но значение 0 у 11%,
- модель: 250 вариантов, 5% пропусков,
- пробег: от 5000 до 150 000, нет пропусков,
- месяц регистрации: нет пропусков, но есть значение - 0,
- тип топлива: 7 вариантов, 9% пропусков,
- марка: 40 вариантов, нет пропусков,
- ремонт: boolean, 20% пропусков,
- фото: всегда 0,
- в датасете 4 дубликата


<div class="alert alert-block alert-success">
<b>Успех:</b> Использован продвинутый инструмент. Главное во время обучения не злоупотреблять, чтобы руки не забыл как делать EDA анализ)
</div>


In [10]:
# Удалим дубликаты:
autos.drop_duplicates(inplace=True)
autos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 354365 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   DateCrawled        354365 non-null  datetime64[ns]
 1   Price              354365 non-null  int64         
 2   VehicleType        316875 non-null  object        
 3   RegistrationYear   354365 non-null  int64         
 4   Gearbox            334532 non-null  object        
 5   Power              354365 non-null  int64         
 6   Model              334660 non-null  object        
 7   Kilometer          354365 non-null  int64         
 8   RegistrationMonth  354365 non-null  int64         
 9   FuelType           321470 non-null  object        
 10  Brand              354365 non-null  object        
 11  Repaired           283211 non-null  object        
 12  DateCreated        354365 non-null  datetime64[ns]
 13  NumberOfPictures   354365 non-null  int64        

In [11]:
# Удалим 171 строку где год регистрации аномальный (больше 2024 или меньше 1900)
autos = autos[(autos['RegistrationYear']<2024)&(autos['RegistrationYear']>1900)]
autos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 354194 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   DateCrawled        354194 non-null  datetime64[ns]
 1   Price              354194 non-null  int64         
 2   VehicleType        316875 non-null  object        
 3   RegistrationYear   354194 non-null  int64         
 4   Gearbox            334499 non-null  object        
 5   Power              354194 non-null  int64         
 6   Model              334564 non-null  object        
 7   Kilometer          354194 non-null  int64         
 8   RegistrationMonth  354194 non-null  int64         
 9   FuelType           321427 non-null  object        
 10  Brand              354194 non-null  object        
 11  Repaired           283187 non-null  object        
 12  DateCreated        354194 non-null  datetime64[ns]
 13  NumberOfPictures   354194 non-null  int64        

In [12]:
autos['DateCrawled_y'] = autos['DateCrawled'].dt.year
autos['DateCrawled_m'] = autos['DateCrawled'].dt.month

In [13]:
autos['DateCrawled_y']

0         2016
1         2016
2         2016
3         2016
4         2016
          ... 
354364    2016
354365    2016
354366    2016
354367    2016
354368    2016
Name: DateCrawled_y, Length: 354194, dtype: int32

In [14]:
autos.groupby(['DateCrawled_m', 'RegistrationYear'])['Model'].count().reset_index()

,DateCrawled_m,RegistrationYear,Model
0,3,1910,30
1,3,1915,0
2,3,1920,0
3,3,1923,2
4,3,1925,0
...,...,...,...
172,4,2015,196
173,4,2016,1353
174,4,2017,1430
175,4,2018,571


In [15]:
autos[autos['DateCrawled_m']==3]['RegistrationYear'].value_counts()

RegistrationYear
2000    20599
1999    19065
2005    18553
2001    16801
2006    16574
        ...  
1927        1
1925        1
1915        1
1949        1
1946        1
Name: count, Length: 96, dtype: int64

In [16]:
autos[autos['DateCrawled_m']==4]['RegistrationYear'].value_counts()

RegistrationYear
2000    3891
1999    3662
2005    3556
2006    3326
2001    3322
        ... 
1948       1
1931       1
1947       1
1936       1
1933       1
Name: count, Length: 81, dtype: int64

<div class="alert alert-block alert-info">
<b>Совет:</b> Советую посмотреть на дату выгрузки анкет - там тоже можно увидеть полезную информацию по поводу границ года регистраци.
</div>



<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Не нашла связи между датой выгрузки (год и месяц) и датой регистрации машины. Анкеты выгружали в марте и апреле 2016, в апреле меньше (16% после удаления дубликатов), но машины с годом регистрации около 1900 есть и там и там. Топ по количеству машин в разные годы регистрации тоже выглядит похоже. 
</div>


<div class="alert alert-block alert-success">
<b>Ответ[2]:</b> Согласно дате выгрузки базы, у нас не может быть год регистрации больше 2016 года 
</div>

In [17]:
autos[autos['Power']==0].head(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen,DateCrawled_y,DateCrawled_m
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24,0,70435,2016-04-07 03:16:57,2016,3
15,2016-03-11 21:39:15,450,small,1910,NaN,0,ka,5000,0,petrol,ford,NaN,2016-03-11,0,24148,2016-03-19 08:46:47,2016,3
32,2016-03-15 20:59:01,245,sedan,1994,NaN,0,golf,150000,2,petrol,volkswagen,no,2016-03-15,0,44145,2016-03-17 18:17:43,2016,3
37,2016-03-28 17:50:15,1500,NaN,2016,NaN,0,kangoo,150000,1,gasoline,renault,no,2016-03-28,0,46483,2016-03-30 09:18:02,2016,3
40,2016-03-26 22:06:17,0,NaN,1990,NaN,0,corsa,150000,1,petrol,opel,NaN,2016-03-26,0,56412,2016-03-27 17:43:34,2016,3
54,2016-03-17 07:56:40,4700,wagon,2005,manual,0,signum,150000,0,NaN,opel,no,2016-03-17,0,88433,2016-04-04 04:17:32,2016,3
55,2016-03-26 10:39:35,550,wagon,1999,manual,0,astra,150000,8,gasoline,opel,yes,2016-03-26,0,56759,2016-04-01 23:17:27,2016,3
69,2016-04-01 11:56:40,1200,coupe,2001,NaN,0,astra,150000,0,NaN,opel,NaN,2016-04-01,0,47249,2016-04-07 08:46:07,2016,4
70,2016-03-08 01:36:42,800,small,1993,manual,0,polo,150000,3,petrol,volkswagen,no,2016-03-08,0,8258,2016-04-05 23:46:00,2016,3
90,2016-03-23 11:53:21,2400,sedan,2003,manual,0,a4,150000,9,gasoline,audi,NaN,2016-03-23,0,40210,2016-03-23 11:53:21,2016,3


In [18]:
# Функция для замены 0 на медианное значение
def replace_zero_with_median(group):
    median_value = group['Power'].median()
    group['Power'] = group['Power'].replace(0, median_value)
    return group

# Применяем функцию к группам по Brand и Model
autos = autos.groupby(['Brand', 'Model'], group_keys=False).apply(replace_zero_with_median)

/tmp/ipykernel_307/2473047989.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  autos = autos.groupby(['Brand', 'Model'], group_keys=False).apply(replace_zero_with_median)





<div class="alert alert-block alert-info">
<b>Совет:</b>  А очень большие мощности кажутся адекватными значениям? И вопрос на подумать: Как думаешь а машины с 0-ой мощностью могут быть просто без двигателя?
</div>


In [19]:
len(autos[autos['Power']>700])

299

In [20]:
autos = autos[autos['Power']<700]
autos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 334263 entries, 0 to 354368
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   DateCrawled        334263 non-null  datetime64[ns]
 1   Price              334263 non-null  int64         
 2   VehicleType        303737 non-null  object        
 3   RegistrationYear   334263 non-null  int64         
 4   Gearbox            318645 non-null  object        
 5   Power              334263 non-null  float64       
 6   Model              334263 non-null  object        
 7   Kilometer          334263 non-null  int64         
 8   RegistrationMonth  334263 non-null  int64         
 9   FuelType           308656 non-null  object        
 10  Brand              334263 non-null  object        
 11  Repaired           272352 non-null  object        
 12  DateCreated        334263 non-null  datetime64[ns]
 13  NumberOfPictures   334263 non-null  int64        

<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Удалила машины с мощностью больше 700, кажется ошибкой, тк это вроде максимальная мощность на рынке. С 0 мощностью поискала объявления на авто.ру: объявления без двигателя есть, но мощность указана. Может с налогами связано, которые по мощности расчитаны в том числе, и это повлияет на цену продажи?
</div>

<div class="alert alert-block alert-success">
<b>Успех[2]:</b>  тут еще напрашивается вопрос: нужны ли автомобили, которые подойдут в категорию запчасти, нашему сервису 'не бит, не крашен' для прогнозирования цен на нормальные автомобили?)
</div>


In [21]:
autos.loc[[0]]

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen,DateCrawled_y,DateCrawled_m
0,2016-03-24 11:52:17,480,NaN,1993,manual,95.0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24,0,70435,2016-04-07 03:16:57,2016,3


In [22]:
# Проверим VW Golf
autos[autos['Model']=='golf'].sample(3)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen,DateCrawled_y,DateCrawled_m
40033,2016-03-16 23:38:34,2500,sedan,1999,manual,150.0,golf,150000,11,petrol,volkswagen,no,2016-03-16,0,64354,2016-04-06 00:17:55,2016,3
230033,2016-03-12 16:58:27,3400,sedan,2007,manual,80.0,golf,150000,4,petrol,volkswagen,no,2016-03-12,0,72820,2016-03-30 16:46:57,2016,3
26261,2016-03-22 16:55:33,8100,small,2009,manual,122.0,golf,125000,7,petrol,volkswagen,no,2016-03-22,0,91054,2016-03-22 17:40:50,2016,3


In [23]:
# Заполним пропуски в категориальных переменных заглушкой
columns_to_fill = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Repaired']
autos[columns_to_fill] = autos[columns_to_fill].fillna('other')

<div class="alert alert-block alert-success">
<b>Успех:</b>  Пропуски обработаны хорошим образом
</div>


<div class="alert alert-block alert-info">
<b>Совет:</b> У fuel_type есть категории, которые означают одно и тоже - их можно объединить, либо подумать может они действительно означают, что-то разное.
</div>


<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Объединила бензин
</div>


<div class="alert alert-block alert-success">
    
<b>Успех[2]:</b> 👍
</div>


In [24]:
autos['FuelType'].unique()

array(['petrol', 'gasoline', 'other', 'lpg', 'hybrid', 'cng', 'electric'],
      dtype=object)

In [25]:
autos['FuelType'] = autos['FuelType'].replace('gasoline', 'petrol')
autos['FuelType'].unique()

array(['petrol', 'other', 'lpg', 'hybrid', 'cng', 'electric'],
      dtype=object)

<div class="alert alert-block alert-info">
<b>Совет:</b>  А около-нулевые цены, не кажется подозрительными? Самое интересное, что на нескольких сайтов по продажам подержанных автомобилей в Германии (страну можно определить по почтовым индексам, большинство из них из Германии) показало, что цены начинаются действительно с 0 и 1 евро, но это единичные объявления и не понятно, то ли это ошибка при заполнении формы, то ли машины на металлолом. Но я  все-таки склоняюсь к тому, чтобы избавиться от таких данных
</div>


<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Удалила объявления дешевле 10 евро
</div>


<div class="alert alert-block alert-success">
    
<b>Успех[2]:</b> Есть
</div>


In [26]:
autos[autos['Price']<10]['PostalCode'].value_counts()

PostalCode
15344    81
27432    25
58840    25
52249    24
15345    23
         ..
66123     1
85399     1
9244      1
18442     1
22559     1
Name: count, Length: 3865, dtype: int64

In [27]:
len(autos[autos['Price']<10])

9534

In [28]:
autos = autos[autos['Price']>10]

In [29]:
# Удалим для построения модели неинформативные колонки или с низкой корреляцией с таргетом
autos = autos.drop(columns=['NumberOfPictures', 'DateCrawled', 'DateCreated', 'LastSeen', 'PostalCode', \
                            'DateCrawled_y', 'DateCrawled_m'], errors='ignore')


<div class="alert alert-block alert-success">
<b>Успех:</b> Удалить неинформативные признаки - верное решение
</div>


In [30]:
autos.columns

Index(['Price', 'VehicleType', 'RegistrationYear', 'Gearbox', 'Power', 'Model',
       'Kilometer', 'RegistrationMonth', 'FuelType', 'Brand', 'Repaired'],
      dtype='object')

## Обучение моделей

In [31]:
# Разделяем признаки на категориальные и числовые 
cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']

# Разделяем данные на обучающую, валидационную и тестовую выборки
features = autos.drop('Price', axis=1)
target = autos['Price']
features_train, features_temp, target_train, target_temp = train_test_split(features, \
                                                                            target, \
                                                                            test_size=0.2, \
                                                                            random_state=42)
features_valid, features_test, target_valid, target_test = train_test_split(features_temp, \
                                                                            target_temp, \
                                                                            test_size=0.5, \
                                                                            random_state=42)

# Создаем OrdinalEncoder и обучаем только на обучающей выборке
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
features_train[cat_features] = ordinal_encoder.fit_transform(features_train[cat_features])

# Применяем трансформацию к валидационной и тестовой выборкам
features_valid[cat_features] = ordinal_encoder.transform(features_valid[cat_features])
features_test[cat_features] = ordinal_encoder.transform(features_test[cat_features])




<div class="alert alert-block alert-danger">
<b>Ошибка::</b> 
    
1) LabelEncoder инструмент предназначенный работать с целевым признаком, кодировать его  обучащие признаки - не хорошо и считается ошибкой, лучше использовать OrdinalEncoder
    
2) Но и  OE  здесь был бы - хороший выбор для всех моделей, кроме линейных, они не могут корректно обрабатывать номинальные данные обработанные порядковым способом. Для них хорош OHE, но тот же OHE нельзя применять для всех других моделей здесь (потому что не эффективно, с точки зрения времени)
        
- Сделать 1 набор данных: 
    - Закодировать для всех моделей методом TargetEncoder, BinaryEncoder - вполне универсальные варианты
    - Закодировать все признаки методом OHE, а модель машины методом OE
    - Заменить использование линейных моделей (так как их рассмотрение не обязательно) и использовать единственный метод кодировки OE.
- Сделать 2 набора данных
    - Закодировать для линейных моделей методом OHE, для остальных OE (или внутренний метод кодирования данных)
    
3) Отмечу, что encoder правильно применять после разбиения данных и обучать только на обучающей выборке, для остальных выборок просто использовать transform. Примеры использования с объяснениями можно найти посмотреь https://colab.research.google.com/drive/1_gAMXcQKoCShB_l8FNtYEejMnosm9mvt?usp=sharing 

И не забывай использовать параметр `handle_unknown`
    
4) Тебе также нужна еще третья тестовая выборка - для итогового оценивания лучшей модели


<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Добавила выборку и обновила encoder
</div>


<div class="alert alert-block alert-success">
    
<b>Успех[2]:</b> Все правильно сделала
</div>


In [32]:
# Гиперпараметры для CatBoostRegressor
params_cb = {
    'iterations': [50, 100],
    'depth': [4, 6],
    'learning_rate': [0.01, 0.1]
}

# Запускаем GridSearchCV
grid_search = GridSearchCV(estimator=CatBoostRegressor(loss_function="RMSE", silent=True),
                           param_grid=params_cb,
                           scoring='neg_root_mean_squared_error',
                           cv=3,
                           n_jobs=-1)

grid_search.fit(features_train, target_train)

# Получаем лучшие параметры и лучший результат RMSE
best_score = -grid_search.best_score_  # Конвертировать обратно в положительное значение RMSE
best_params = grid_search.best_params_

# Предсказание на валидационном наборе
preds = grid_search.predict(features_valid)
validation_rmse = np.sqrt(mean_squared_error(target_valid, preds))

print(f'СatBoost Best RMSE (GridSearch): {best_score}')
print(f'СatBoost Best parameters (GridSearch): {best_params}')
print(f'СatBoost Validation RMSE: {validation_rmse}')

СatBoost Best RMSE (GridSearch): 1850.9930117583292
СatBoost Best parameters (GridSearch): {'depth': 6, 'iterations': 100, 'learning_rate': 0.1}
СatBoost Validation RMSE: 1816.9509029838184


In [33]:
# Достаем время обучения и время предсказания для каждой комбинации гиперпараметров
training_times = grid_search.cv_results_['mean_fit_time']
prediction_times = grid_search.cv_results_['mean_score_time']

# Печатаем время обучения и предсказания для каждой комбинации
for param, train_time, pred_time in zip(grid_search.cv_results_['params'], training_times, prediction_times):
    print(f'Parameters: {param}, Training time: {train_time:.4f} seconds, Prediction time: {pred_time:.4f} seconds')

Parameters: {'depth': 4, 'iterations': 50, 'learning_rate': 0.01}, Training time: 1.4836 seconds, Prediction time: 0.0092 seconds
Parameters: {'depth': 4, 'iterations': 50, 'learning_rate': 0.1}, Training time: 1.4185 seconds, Prediction time: 0.0132 seconds
Parameters: {'depth': 4, 'iterations': 100, 'learning_rate': 0.01}, Training time: 2.3712 seconds, Prediction time: 0.0126 seconds
Parameters: {'depth': 4, 'iterations': 100, 'learning_rate': 0.1}, Training time: 2.3312 seconds, Prediction time: 0.0160 seconds
Parameters: {'depth': 6, 'iterations': 50, 'learning_rate': 0.01}, Training time: 1.7199 seconds, Prediction time: 0.0101 seconds
Parameters: {'depth': 6, 'iterations': 50, 'learning_rate': 0.1}, Training time: 1.5816 seconds, Prediction time: 0.0136 seconds
Parameters: {'depth': 6, 'iterations': 100, 'learning_rate': 0.01}, Training time: 2.7093 seconds, Prediction time: 0.0132 seconds
Parameters: {'depth': 6, 'iterations': 100, 'learning_rate': 0.1}, Training time: 2.6577 s

In [34]:
# Получаем время обучения и время предсказания для лучших параметров
best_index = grid_search.best_index_
best_fit_time = grid_search.cv_results_['mean_fit_time'][best_index]
best_score_time = grid_search.cv_results_['mean_score_time'][best_index]

print(f'СatBoost Best training time: {best_fit_time:.4f} seconds')
print(f'СatBoost Best prediction time: {best_score_time:.4f} seconds')

СatBoost Best training time: 2.6577 seconds
СatBoost Best prediction time: 0.0168 seconds


In [35]:
# Гиперпараметры для LightGBM
params_lgbm = {
    'n_estimators': [50, 100],
    'max_depth': [4, 6],
    'learning_rate': [0.01, 0.1]
}

# Запускаем GridSearchCV для LightGBM
grid_search_lgbm = GridSearchCV(estimator=LGBMRegressor(),
                                 param_grid=params_lgbm,
                                 scoring='neg_root_mean_squared_error',
                                 cv=3,
                                 n_jobs=-1)

grid_search_lgbm.fit(features_train, target_train)

# Получаем лучшие параметры и лучший результат RMSE для LightGBM
best_score_lgbm = -grid_search_lgbm.best_score_
best_params_lgbm = grid_search_lgbm.best_params_

# Предсказание на валидационном наборе для LightGBM
preds_lgbm = grid_search_lgbm.predict(features_valid)
validation_rmse_lgbm = np.sqrt(mean_squared_error(target_valid, preds_lgbm))

print(f'LightGBM Best RMSE (GridSearch): {best_score_lgbm}')
print(f'LightGBM Best parameters (GridSearch): {best_params_lgbm}')
print(f'LightGBM Validation RMSE: {validation_rmse_lgbm}')

LightGBM Best RMSE (GridSearch): 1752.6672492609725
LightGBM Best parameters (GridSearch): {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100}
LightGBM Validation RMSE: 1714.110015024773


In [36]:
# Получаем время обучения и время предсказания для лучших параметров
best_index_lgbm = grid_search_lgbm.best_index_
best_fit_time_lgbm = grid_search_lgbm.cv_results_['mean_fit_time'][best_index_lgbm]
best_score_time_lgbm = grid_search_lgbm.cv_results_['mean_score_time'][best_index_lgbm]

print(f'LightGBM Best training time: {best_fit_time_lgbm:.4f} seconds')
print(f'LightGBM Best prediction time: {best_score_time_lgbm:.4f} seconds')

LightGBM Best training time: 3.3339 seconds
LightGBM Best prediction time: 0.6659 seconds


In [37]:
# Гиперпараметры для RandomForest
params_rf = {
    'n_estimators': [50, 100],
    'max_depth': [4, 6],
    'min_samples_split': [2, 5]
}

# Запускаем GridSearchCV для RandomForestRegressor
grid_search_rf = GridSearchCV(estimator=RandomForestRegressor(),
                               param_grid=params_rf,
                               scoring='neg_root_mean_squared_error',
                               cv=3,
                               n_jobs=-1)

grid_search_rf.fit(features_train, target_train)

# Получаем лучшие параметры и лучший результат RMSE для RandomForest
best_score_rf = -grid_search_rf.best_score_
best_params_rf = grid_search_rf.best_params_

# Предсказание на валидационном наборе для RandomForest
preds_rf = grid_search_rf.predict(features_valid)
validation_rmse_rf = np.sqrt(mean_squared_error(target_valid, preds_rf))

print(f'RandomForest Best RMSE (GridSearch): {best_score_rf}')
print(f'RandomForest Best parameters (GridSearch): {best_params_rf}')
print(f'RandomForest Validation RMSE: {validation_rmse_rf}')

RandomForest Best RMSE (GridSearch): 2260.8488104350345
RandomForest Best parameters (GridSearch): {'max_depth': 6, 'min_samples_split': 2, 'n_estimators': 100}
RandomForest Validation RMSE: 2236.8186258989035


In [38]:
# Получаем время обучения и время предсказания для лучших параметров
best_index_rf = grid_search_rf.best_index_
best_fit_time_rf = grid_search_rf.cv_results_['mean_fit_time'][best_index_rf]
best_score_time_rf = grid_search_rf.cv_results_['mean_score_time'][best_index_rf]

print(f'RandomForest Best training time: {best_fit_time_rf:.4f} seconds')
print(f'RandomForest Best prediction time: {best_score_time_rf:.4f} seconds')

RandomForest Best training time: 14.6240 seconds
RandomForest Best prediction time: 0.3675 seconds



<div class="alert alert-block alert-danger">
<b>Ошибка:</b>
    
- время обучения это  чистый `.fit()` модели - без подбора гиперпараметров и без предсказаний,  то есть время обучения gridsearch не подойдет

p.s. можно вытаскивать  все  метрики  интересующие заказчика лаконично из GridSearchCV/RandomizedSearchCV, все они лежат в `.cv_results_`
</div>

<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Обновила с учетом новой выборки, поменяла модели и вывела время из .cv_results_
</div>


<div class="alert alert-block alert-success">
    
<b>Успех[2]:</b> Лаконичный выбор
</div>


## Анализ моделей

Исходя из представленных результатов по трем моделям (CatBoost, LightGBM и RandomForest), можно сделать следующие выводы:

1. Качество предсказания (RMSE):
   - CatBoost: Best RMSE - 1850.99 (Validation RMSE - 1816.95)
   - LightGBM: Best RMSE - 1752.67 (Validation RMSE - 1714.11)
   - RandomForest: Best RMSE - 2259.38 (Validation RMSE - 2243.93)

2. Временные характеристики:
   - CatBoost: Training time - 2.82 seconds, Prediction time - 0.017 seconds
   - LightGBM: Training time - 3.43 seconds, Prediction time - 0.664 seconds
   - RandomForest: Training time - 7.66 seconds, Prediction time - 0.196 seconds

Выбор модели:
На основе критериев заказчика, мы можем выбрать LightGBM, так как она имеет наименьший RMSE на валидационной выборке (1714.11), что значительно ниже порога в 2500. Несмотря на то, что CatBoost имеет лучшее время обучения, качество предсказания у LightGBM выше.

Теперь проверим качество модели LightGBM на тестовой выборке:


<div class="alert alert-block alert-success">
<b>Успех[2]:</b> Отлично проделан шаг
    
- Радует количество рассмотренных моделей.   
- Кросс-валидационные методы успешно использованы. 
- Все метрики интересующие заказчика корректно вычислены и проанализированы. 
- Выбор лучшей модели обоснован. 
</div>


In [39]:
# Инициализация модели с лучшими параметрами
best_lightgbm = LGBMRegressor(learning_rate=0.1, max_depth=6, n_estimators=100)

# Обучение модели на тренировочных данных
best_lightgbm.fit(features_train, target_train)

# Предсказание на тестовых данных
test_preds = best_lightgbm.predict(features_test)

# Вычисление RMSE на тестовой выборке
test_rmse = np.sqrt(mean_squared_error(target_test, test_preds))

print(f'Test RMSE for LightGBM: {test_rmse:.4f}')

Test RMSE for LightGBM: 1727.6239


<div class="alert alert-block alert-success">
<b>Успех[2]:</b> На тестовой выборке получено хорошее качество!
</div>


<div class="alert alert-block alert-danger">
<b>Ошибка:</b> После анализа и выбора одной наилучшей модели, должно идти  ее тестирование. Другие модели тестовую выборку использовать не должны.
</div>

<div class="alert alert-block alert-info">
<b>Совет:</b> 

Также если говорить, что можно ещё улучшить в подобных проектах, то я бы выделил такие моменты:<br>
    
1) Для понимания, а какие в итоге факторы важны при моделировании, можно выводить их важность, использую feature_importances_, ну и график заодно. Это будет смотреться очнь здорово: результативная метрика и график важности факторов)))
    
2) У нас разный возраст машин. Есть гипотеза, что для разных возрастов - своё ценообразование. Поэтому, можно попробовать ввести фактор "тип возраста" (ретро, супер-ретро, старая, новая... надо подумать..)..<br>

3) У некоторых моделей семейства бустингов, есть внутренний метод кодировки данных, который хорошо было бы попробовать
</div>



<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Обновила вывод, протестировала лучшую модель на тестовой выборке
</div>

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей